Team submitting this assignment:  
_<b>Sindhu Ranga, Disha Jain, Sameer Gupta, Jason Quinn</b>_

External resources used:  
_it is not necessary to list the course materials, but if you used any other resources, including discussing problems with students not on your team, list them here_

## Project 3: Matchmaker, Matchmaker

<div class="alert alert-block alert-danger" align="center">
    <b>Due: <font color="red">9:59pm</font>, Tuesday, 18 February 2020<b> (note the later than normal due time!)
</div>

This project focuses on the allocation of discrete goods without the use of money. 

This project is based on kidney donations in the United States. The National Organ Transplant Act of 1984 outlawed the purchase and sale of kidneys, preventing a pricing-based market. Since the early 2000s, economists and medical professionals have developed different solutions to the problem of allocating kidneys from donors to recipients. There's an NPR 
[_Hidden Brain_ Podcast Episode](https://www.npr.org/2019/02/27/698563807/for-sale-by-owner-the-psychology-of-repugnant-transactions) featuring Alvin Roth speaking with Shankar Vedantam on this topic!

This project will guide you through the basics of discrete exchanges, and you will propose and implement your own method at the end. After receiving your submissions, we will run a competition evaluating all the team submissions on a (secret, until after the competition) test dataset.

For this assignment, you should form your own team with the following constraints:

- It must be three or four people.

- At least one team member must be in the Economics section; at least one team member must be in the Computer Science section.

It is fine to continue with the same team you have worked with on a previous project. If you have problems forming a team that you are happy with, contact the course staff. You are encouraged to use the `#teaming` channel in the course slack to find teammates.

### Kidney Exchange

Humans are born with two kidneys, but can live well with just one healthy kidney. Hence, there is an opportunity for a live donor to donate a kidney to a patient whose kidneys have failed. 

There are two important factors when considering recipient and donor compatibility: blood type and tissue type. A recipient, $R$, and a donor, $D$, must pass both a blood type and tissue type compatibility test to be considered compatible for a transplant. 

1.  Blood Type Compatibility: The four blood types are <b>O</b>, <b>A</b>, <b>B</b>, and <b>AB</b>.

    -   Type <b>O</b> is compatible with all other types (that is, a kidney from a donor with type **O** can be matched with a recipient with any blood type).

    -   Type <b>A</b> is compatible with types <b>A</b> and **AB**.

    -   Type <b>B</b> is compatible with types <b>B</b> and **AB**.

    -   Type **AB** is compatible with type **AB**.
    

2.  Tissue Compatibility: Tissue compatibility depends on the compatibility of immune system markers (HLA) between the donor and recipient. Roughly, the closeness of the HLA match required depends on the recipient's reactve immune system, measured as *percent reactive antibody* (PRA). For this project we discretize these values:

    -   _Low-PRA_ individuals (about 70% of population) have less active antibodies. A _Low-PRA_ recipient has a 95% chance of being tissue-compatible with a randomly-selected donor.

    -   _Medium-PRA_ individuals (about 20% of population) have a 55% chance of being tissue-compatible as a recipient for a randomly-selected donor.

    -   _High-PRA_ individuals (about 10% of population) have a 10% changes of being tissue-compatible as a recipient for a randomly-selected donor.

We will ignore the biological issues in whether or not a donor and recipient are compatible, and provide code that simulates this compatibility test with the expected probabilities.

These tests are independent of one another. In order for a donor kidney to be acceptable, it must be both _blood type compatible_ and _tissue-type compatible_ with the recipient.

### Modeling Kidney Exchange

The goal of a kidney exchange is to enable patients in need of a kidney who have someone willing to donate to them, but whose kidney is incompatible, find a chain of donor-recipients that enable the patient to receive a compatible kidney from another donor in exchange for this patient's willing donor donating a kidney to another patient.

It is common to organize the market in the following way. Each recipient, $R_i$, is paired with their incompatible but willing donor, $D_i$. A market of size $N$ will have $N$ incompatible pairs {$(R_1, D_1), ..., (R_n, D_n)$}. A donor will donate her kidney if and only if her paired but incompatible recipient also receives a kidney. 

We can model the market is a directed graph where each edge connects a donor with a compatible recipient. Thus, the edges represent all directly compatible exchanges. The problem the market designer faces is finding a method that will maximize the survival rate while accounting for the constraints on donations.

Donation is usually done through _k_-way exchanges, which can be represented as cycles in the directed graph.

A _two-way exchange_ is when a donor, $D_i$, donates to a recipient, $R_j$, whose incompatible partner $D_j$ then donates to $R_i$. For this exchange to work, $D_i$ must be compatible with $R_j$ and $D_j$ must be compatible with $R_i$. There are a total of two incompatible pairs, two donors and two recipients, in a two-way exchange. In a _three-way exchange_, the donor in pair 1 donates to pair 2, whose donor donates to pair 3, whose donor donates to pair 1. A _k-way exchange_ would have $k$ pairs donating in a cycle of length $k$.

For more information, see [_Efficient Kidney Exchange: Coincidence of Wants in Markets with Compatibility-Based Preferences_](http://uvammm.github.io/docs/kidneyexchange.pdf) by Alvin E. Roth, Tayfun Sönmez and M. Utku Ünver, American Economic Review 2007. 

### Data

Since the medical data such as what is needed for kidney exchange is sensitive, we will use simulated data. We have provided some simulated data in [`patients.csv`](https://raw.githubusercontent.com/uvammm/uvammm.github.io/master/projects/project3/patients.csv) drawn from the empirical distribution of characteristics. You should download this file, and save it in the directory where you are working on the notebook.

The code we used to generate that data (and the other code included in this notebook) is here: [`simulator.py`](https://raw.githubusercontent.com/uvammm/uvammm.github.io/master/projects/project3/simulator.py) (you are welcome to run or modify that code to produce your own data, but the results you report for the problems should be done using the provided data in the [`patients.csv`](https://raw.githubusercontent.com/uvammm/uvammm.github.io/master/projects/project3/patients.csv) file).

Each row represents an incompatible pair with the following fields:

- `TimePeriod`: The time period when the pair entered the kidney exchange pool
- `ReceiverID`: A unique number representing the receiver
- `ReceiverBloodType`: The receiver's blood type (`A`, `B`, `O` or `AB`)
- `ReceiverPRA`: The receiver's PRA (`Low`, `Medium` or  `High`)
- `ReceiverSurvivalPrb`: The probability that an unmatched receiver will survive till the next time period.
- `DonorID`: A unique number representing the donor
- `DonorBloodType`: The donor's blood type (`A`, `B`, `O` or `AB`)

Here is some sample code for reading that file:

In [2]:
import csv
def read_dataset(f):
    """                                                                                                                                      
    Reads a csv file of patient records, and returns that dataset as a list.                                                                 
    This is very fragile (non-defensive) code that assumes the csv is correct,                                                               
    and converts to the appropriate types.                                                                                                   
    """
    patients = [] # list of (patient, donor) pairs                                                                                           
    with open(f, newline='') as csvfile:
        simreader = csv.reader(csvfile, delimiter=',')
        headers = next(simreader)
        print(str(headers))
        for row in simreader:
            readline = {key: value for key, value in zip(headers, row)}
            patients.append({'TimePeriod': int(readline['TimePeriod']),
                             'ReceiverID': int(readline['ReceiverID']),
                             'ReceiverBloodType': readline['ReceiverBloodType'],
                             'ReceiverPRA': readline['ReceiverPRA'],
                             'ReceiverSurvivalPrb': float(readline['ReceiverSurvivalPrb']),
                             'DonorID': int(readline['DonorID']),
                             'DonorBloodType': readline['DonorBloodType']})

    return patients

In [2]:
pairs = read_dataset('patients.csv')
print ("Number of patients: " + str(len(pairs)))
# print ("blood type of patient in the 37th friend-pair: " + pairs[37]['ReceiverBloodType'])
# print ("donorID of the 598th friend-pair: " + str(pairs[598]['DonorID']))

['TimePeriod', 'ReceiverID', 'ReceiverBloodType', 'ReceiverPRA', 'ReceiverSurvivalPrb', 'DonorID', 'DonorBloodType']
Number of patients: 1000



We have provided two functions that you can use to test for blood type, `is_blood_compatible`, and tissue type `is_tissue_compatible` compatibility.

In [3]:
def is_blood_compatible(receiver_type, donor_type):
    """                                                                                                                                      
    Returns True iff a receiver with blood time receiver_type can receive blood                                                              
    from a donor with blood type donor_type.                                                                                                 
    """
    if donor_type == 'O':
        return True
    elif donor_type == 'A':
        return receiver_type in ('A', 'AB')
    elif donor_type == 'B':
        return receiver_type in ('B', 'AB')
    elif donor_type == 'AB':
        return receiver_type == 'AB'
    else:
        raise InvalidParameterException("Invalid blood type: " + donor_type)

def is_tissue_compatible(recv_pra, recv_id, don_id):
    """                                                                                                                                      
    Modeling actual tissue compatibility is complex, and depends on                                                                          
    properties of different HLA markers and various other complications.                                                                     
    Instead of dealing with the medical complexities, we use a simple                                                                        
    model that produces a uniformly-distributed value that is dependent                                                                      
    on the two inputs, and outputs a discretized probability.                                                                                
                                                                                                                                             
    It's not important to understand the following code. But you should                                                                      
    call this function with the receiver's PRA-type, receiver's tissue ID,                                                                   
    and the donor's ID to check if their tissues are compatible or not.                                                               
                                                                                                                                             
    Example usage: is_tissue_compatible('Low', 4474, 3587)                                                                                   
    """
    import hashlib
    
    # This code is a convoluted way to approximate a random oracle on the tissue ids (so the                                                 
    # output is uniformly random in [0, 1), but for a given tissue pair, the same output is always                                           
    # produced).                                                                                                                             

    hexval = hashlib.md5((str(recv_id) + str(don_id)).encode()).hexdigest()
    intval = int(hexval, 16)
    b = intval / (1 << 129 - 1)  # md5 generates 128-bit number; we normalize it to [0, 1]                                                   

    if recv_pra == 'Low':
        return b <= 0.95
    elif recv_pra == 'Medium':
        return b <= 0.45
    else:  # high pra                                                                                                                        
        assert recv_pra == 'High'
        return b <= 0.10

def is_pair_compatible(receiver, donor):
    """                                                                                                                                      
    This function returns True if the receiver from the receiver pair is compatible with                                                     
    the donor from the donor pair, otherwise it returns False.                                                                               
    """
    return is_blood_compatible(receiver_type=receiver['ReceiverBloodType'], donor_type=donor['DonorBloodType']) \
        and is_tissue_compatible(recv_pra=receiver['ReceiverPRA'], recv_id=receiver['ReceiverID'], don_id=donor['DonorID'])

In [4]:
pairs = read_dataset('patients.csv')

for i in range(50):
    if is_pair_compatible(pairs[0], pairs[i]):
        print("pair 0 is compatible with pair", i)
is_pair_compatible(pairs[0], pairs[12])

['TimePeriod', 'ReceiverID', 'ReceiverBloodType', 'ReceiverPRA', 'ReceiverSurvivalPrb', 'DonorID', 'DonorBloodType']
pair 0 is compatible with pair 1
pair 0 is compatible with pair 3
pair 0 is compatible with pair 5
pair 0 is compatible with pair 6
pair 0 is compatible with pair 7
pair 0 is compatible with pair 10
pair 0 is compatible with pair 11
pair 0 is compatible with pair 12
pair 0 is compatible with pair 13
pair 0 is compatible with pair 14
pair 0 is compatible with pair 15
pair 0 is compatible with pair 17
pair 0 is compatible with pair 18
pair 0 is compatible with pair 19
pair 0 is compatible with pair 22
pair 0 is compatible with pair 23
pair 0 is compatible with pair 24
pair 0 is compatible with pair 27
pair 0 is compatible with pair 30
pair 0 is compatible with pair 31
pair 0 is compatible with pair 32
pair 0 is compatible with pair 33
pair 0 is compatible with pair 34
pair 0 is compatible with pair 35
pair 0 is compatible with pair 36
pair 0 is compatible with pair 37
pair

True

In [6]:
is_pair_compatible(pairs[37], pairs[37])

False

## Warm-up: Static Two-Way Exchange

In the first part of the assignment, you will clear the kidney exchange market using only two-way exchanges. The steps below will guide you through the process of computing a solution to the matching problem.

We will take the given data as static, ignoring the `TimePeriod` and assuming all pairs are present and ready to be matched. Since the matching will occur over a single period, we will also ignore the `ReceiverSurvivalPrb`. Therefore, we will have a market with 599 pairs of agents.

   <div class="alert alert-block alert-warning">
    <b>Problem 1.</b> Compute the mutual compatibility matrix, $\textbf{C}$, for time period 1, where $c_{i,j}=1$ if recipient $i$ is compatible with donor $j$ and zero otherwise. You need to account for blood type and tissue type compatilbility when determining overall compatilbility of two agents. 
    </div>

In [5]:
# Write your code here
import numpy as np
n = len(pairs)

C = np.zeros((n,n))

for i in range (0, n):
    for j in range (i, n):
        if is_pair_compatible(pairs[i], pairs[j]) and is_pair_compatible(pairs[j], pairs[i]):
            C[i][j] = 1
            C[j][i] = 1

   <div class="alert alert-block alert-warning">

<b>Problem 2.</b> Find the maximum matching given $\textbf{C}$ if only two-way exchanges are allowed. For this, you may find the [_Top Trading Cycles algorithm_](https://people.cs.umass.edu/~sheldon/teaching/mhc/cs312/2014sp/Slides/top-trading-cycles.pdf) helpful. Your code should report total number of matched pairs. The target number of matched pairs should be around 320.
    </div>

In [6]:
# Write your code here
# This is not the "maximum", it is simply one possibility
matched = []
matches = 0

for i in range(0, len(pairs)):
    if i not in matched:
        for j in range(i+1, len(pairs)):
            if j not in matched:
                if C[i][j] == 1 and C[j][i] == 1:
                    matches += 1
                    matched.append(i)
                    matched.append(j)
                    break
print(matches)

278


In [7]:
import numpy as np
import copy
import time

In [10]:

import time
#greedy algorithm choice --> start with (rec, donor) pair with least number of both-way matches
import numpy as np
n = len(pairs)

C = np.zeros((n,n))

for i in range (0, n):
    for j in range (i, n):
        if is_pair_compatible(pairs[i], pairs[j]) and is_pair_compatible(pairs[j], pairs[i]):
            C[i][j] = 1
            C[j][i] = 1
n = len(pairs)

all_sums = np.zeros(n)

my_c = copy.deepcopy(C)

def find_sum(num):
    return np.sum(my_c[num])
         
def find_all_sum():
    for i in range (0, n):
        all_sums[i] = find_sum(i)
    return all_sums

def find_min():
    mini = n
    mindex = -1
    for i in range (0, n):
        if mini >= all_sums[i] and all_sums[i] > 0:
            mini = all_sums[i]
            mindex = i
    return mindex

def find_first(num):
    minimum = n
    mindex = -1
    for j in range (0, n):
        if my_c[num][j] == 1.0 and my_c[j][num] == 1.0 and minimum > all_sums[j]:
            minimum = all_sums[j]
            mindex = j
    return mindex
    
def set_zero(num):
    for i in range(0, n):
        my_c[num][i] = 0
        my_c[i][num] = 0

def not_all_zero():
    for i in range (0, n):
        if all_sums[i] != 0:
            return True
    return False
        
matches = 0
match = []

my_t = time.perf_counter()
all_sums = find_all_sum()
pair_one = find_min()
while not_all_zero():
    pair_two = find_first(pair_one)
    match.append((pair_one, pair_two))
    matches += 2
    set_zero(pair_two)
    set_zero(pair_one)
    all_sums = find_all_sum()
    pair_one = find_min()
dif = time.perf_counter() - my_t
print(dif)
print(matches)
print (match)
done = []

my_bool = True
for pair in match:
    i, j = pair
    my_bool = my_bool and is_pair_compatible(pairs[i], pairs[j]) and is_pair_compatible(pairs[j], pairs[i]) and i not in done and j not in done
    done.append(i)
    done.append(j)
    
print(my_bool)

4.060889500000002
576
[(959, 428), (663, 962), (112, 362), (837, 83), (268, 281), (844, 442), (756, 435), (164, 99), (113, 7), (864, 984), (692, 463), (974, 628), (214, 437), (811, 1), (677, 775), (397, 380), (14, 935), (742, 754), (147, 961), (626, 722), (329, 958), (866, 227), (165, 580), (49, 774), (950, 656), (825, 237), (836, 662), (800, 967), (785, 260), (715, 737), (703, 524), (671, 578), (805, 583), (893, 171), (699, 999), (740, 829), (532, 982), (957, 738), (647, 896), (494, 510), (441, 901), (356, 339), (355, 876), (810, 131), (69, 280), (204, 941), (360, 593), (166, 372), (632, 341), (163, 685), (499, 259), (387, 538), (126, 865), (96, 305), (987, 55), (939, 710), (809, 27), (802, 559), (972, 121), (887, 58), (807, 923), (996, 218), (723, 753), (576, 81), (991, 73), (550, 881), (911, 458), (257, 139), (874, 122), (854, 124), (786, 495), (781, 22), (669, 253), (314, 318), (784, 284), (562, 484), (194, 990), (856, 505), (704, 394), (432, 92), (886, 226), (827, 289), (596, 310)

   <div class="alert alert-block alert-warning">

<b>Problem 3.</b> Analyze the execution cost of your matching algorithm (for Problem 2). A good answer will include both 
an asymptotic analysis, and a concrete discussion of how large an exchange problem it could reasonably scale to support.

</div>

_Write your analysis here_

Our greedy algorithm has an upper boung of n^3. In order to perform on 1000 calculations on this machine took almost 5 seconds. Therefore, in order to run this algorithm on every single human being on earth (~ 7.8 billion), it would take around 39 million seconds, or 1.24 years to complete at this rate.



## Three-Way Exchange

For the second part of the assignment, you will extend your matching algorithm to allow both two- and three-way exchanges. You may also use $k$-way exchanges where $k\geq4$. 
For this question, you will continue to use $\textbf{C}$ from problem 1 in order to compare your results from both parts. (We will again take the given data as static, ignoring the `TimePeriod` and assuming all pairs are present and ready to be matched, just as in problem 1.)

<div class="alert alert-block alert-warning">

<b>Problem 4.</b> Find the maximum matching given $\textbf{C}$ from Problem 1 and using two- and three-way exchanges (and optionally, support exchanges up to higher $k$. You will need to modify your code from the previous part to allow for cycles of up to 3 (or more) pairs. Your code should report the total number of matched pairs (not the number of cycles). You should see an increase of about 10% from your earlier two-way exchange.
</div>

In [8]:
# Write your code here
import numpy as np
n = len(pairs)
new_C = np.zeros((n,n))

for i in range (0, n):
    for j in range (0, n):
        if is_pair_compatible(pairs[i], pairs[j]):
            new_C[i][j] = 1
print(new_C)

[[0. 1. 0. ... 1. 0. 1.]
 [1. 0. 0. ... 0. 0. 1.]
 [0. 1. 0. ... 1. 1. 1.]
 ...
 [1. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
# Write your code here

import numpy as np
import copy

def find_sum(num):
    return np.sum(my_c[num])
         
def find_all_sum():
    for i in range (0, n):
        all_sums[i] = find_sum(i)
    return all_sums

def find_min():
    mini = n
    mindex = -1
    for i in range (0, n):
        if mini >= all_sums[i] and all_sums[i] > 0:
            mini = all_sums[i]
            mindex = i
    return mindex

def find_first(num):
    minimum = n
    mindex = -1
    for j in range (0, n):
        if my_c[num][j] == 1.0 and my_c[j][num] == 1.0 and minimum > all_sums[j]:
            minimum = all_sums[j]
            mindex = j
    return mindex
    
def set_zero(num):
    for i in range(0, n):
        my_c[num][i] = 0
        my_c[i][num] = 0

def not_all_zero():
    for i in range (0, n):
        if all_sums[i] != 0:
            return True
    return False


def compute_indegrees(indegrees):
    for receiver in range(0,n):
        for donor in range(0,n):
            indegrees[donor] += new_C[receiver][donor]
        return indegrees

def compute_comp_donor(receiver):
    sum = 0
    for cell in receiver:
        sum += cell
    return sum

def compute_comp_donors(comp_donors):
    for receiver in range(0,n):
        comp_donors[receiver] = compute_comp_donor(new_C[receiver])
    return comp_donors
        
def min_index_of_array(num_array):
    min_value = float('inf')
    mindex = []
    for i in range(0,len(num_array)):
        if num_array[i] <= min_value and num_array[i] > 0 and found_no_matches[i] == 0:
            if num_array[i] < min_value:
                mindex = []
                min_value = num_array[i]
            mindex.append(i)
    return mindex

def comp_donors_present(comp_donors):
    for comp_donor in comp_donors:
        if comp_donor > 0:
            return True
    return False

def update_matrix(removed_pairs):
    for pair in removed_pairs:
        for i in range(0,n):
            new_C[pair][i] = 0
            new_C[i][pair] = 0

def get_compat_donors(receiver_index):
    donors = []
    for i in range(n):
        if new_C[receiver_index][i] == 1:
            donors.append(i)
    return donors

def get_optimal_cycle(cycles, comp_donors, comp_receivers):
    if len(cycles) == 1:
        return cycles[0]
    min_matches = float('inf')
    min_matches_index = 0
    sum_of_donor_matches = np.zeros(len(cycles))
    for i in range(len(cycles)):
        for d in range(1,len(cycles[i])):
            sum_of_donor_matches[i] += comp_receivers[cycles[i][d]]
        if sum_of_donor_matches[i] < min_matches:
            min_matches = sum_of_donor_matches[i]
            min_matches_index = i
    return cycles[i]
        
def calculate_k_cycles(new_C):
    comp_donors = np.zeros(n)
    comp_receivers = np.zeros(n)
    matched_pairs = 0
    matches = []
    # Calc the number of compatible donors for each receiver
    comp_donors = compute_comp_donors(comp_donors)
    comp_receivers = compute_indegrees(comp_receivers)
    # Find the receivers with the fewest numbers of compatible donors
    mindecies = min_index_of_array(comp_donors)
    while mindecies:
        # print(mindecies)
        # get first receiver in mindecies
        mindex = mindecies[0]
        # get all immediate matching donors for receiver
        possible_donors = get_compat_donors(mindex)
        if possible_donors:
            cycles = []
            # for each immediate matching donor
            for donor in possible_donors:
                # if receiver's donor is not a match for donor's reciver, look another level down
                # get compatible donors for donor's receiver, add all cycles where 2nd donor's 
                # patient is a match for first receiver's donor
                possible_donors_2 = get_compat_donors(donor)
                if possible_donors_2:
                    for donor_2 in possible_donors_2:
                        if new_C[donor_2][mindex] == 1:
                            cycles.append((mindex, donor, donor_2))
                else:
                    found_no_matches[mindex] = 1
            if cycles:
                optimal_cycle = get_optimal_cycle(cycles, comp_donors, comp_receivers)
                matches.append(optimal_cycle)
                update_matrix(optimal_cycle)
                matched_pairs += len(optimal_cycle)
            else:
                found_no_matches[mindex] = 1
        else:
            found_no_matches[mindex] = 1

        comp_donors = compute_comp_donors(comp_donors)
        comp_receivers = compute_indegrees(comp_receivers)
        mindecies = min_index_of_array(comp_donors)
    return matched_pairs, matches

n = len(pairs)

C = np.zeros((n,n))

for i in range (0, n):
    for j in range (i, n):
        if is_pair_compatible(pairs[i], pairs[j]) and is_pair_compatible(pairs[j], pairs[i]):
            C[i][j] = 1
            C[j][i] = 1
n = len(pairs)

all_sums = np.zeros(n)

my_c = copy.deepcopy(C)

match_amt = 0
match = []

all_sums = find_all_sum()
pair_one = find_min()
while not_all_zero():
    pair_two = find_first(pair_one)
    match.append((pair_one, pair_two))
    match_amt += 2
    set_zero(pair_two)
    set_zero(pair_one)
    all_sums = find_all_sum()
    pair_one = find_min()


n = len(pairs)
new_C = np.zeros((n,n))

for i in range (0, n):
    for j in range (0, n):
        if is_pair_compatible(pairs[i], pairs[j]):
            new_C[i][j] = 1

found_no_matches = np.zeros(n)
matched_pairs, matches = calculate_k_cycles(new_C)
final_pairs = []
final_pairs_num = 0
for cycle2 in match:
    pair1 = cycle2[0]
    pair2 = cycle2[1]
    cycle_needing_p1 = -1
    cycle_needing_p2 = -1
    for i in range(len(matches)):
        cycle3 = matches[i]
        if pair1 in cycle3:
            cycle_needing_p1 = i
        elif pair2 in cycle3:
            cycle_needing_p2 = i
    if cycle_needing_p2 != -1 and cycle_needing_p1 != -1 and cycle_needing_p2 != cycle_needing_p1:
        match.remove(cycle2)
        final_pairs.append(matches[cycle_needing_p1])
        final_pairs.append(matches[cycle_needing_p2])
    else:
        final_pairs.append(cycle2)
for cycle in final_pairs:
    final_pairs_num += len(cycle)
    
print(final_pairs)
print(final_pairs_num)

In [ ]:
#finding all cycles -- takes too long

cycles = {}
n = len(pairs)
#my_c = [[0, 1, 0, 0, 0, 0],[0, 0, 1, 0, 1, 0], [1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1], [0, 1, 0, 0, 0, 0]]

def all_cycles():
    cycles = {}
    visited = []
    for i in range (0, n):
        dfs(i, i, [], visited)
        visited.append(i)

def dfs(start, v, path, visited):
    if v in visited:
        if start==v:
            cycles[len(cycles)] = path
        return
    visited.append(v)
    for i in range (0, n):
        if new_C[v][i] == 1:
            new_path = copy.deepcopy(path)
            new_path.append(i)
            dfs(start,i,new_path, visited)
    visited.remove(v)

all_cycles()
print(cycles)
print(len(cycles))

In [10]:
cycles = {}
my_mins = {}
saved = [0]

def indegrees(num):
    sum = 0
    for i in range(0, n):
        sum += new_C[num][i]
    return sum

def outdegrees(num):
    sum = 0
    for i in range(0, n):
        sum += new_C[i][num]
    return sum

def get_indegrees(num):
    indegrees = []
    for i in range(0, n):
        if new_C[num][i] == 1:
            indegrees.append(i)
    return indegrees

def find_mins():
    for i in range (0, n):
        my_mins[i] = min(indegrees(i), outdegrees(i))

def find_mini():
    mini = n
    mindex = -1
    for i in range (0, n):
        if my_mins[i] != 0 and mini > my_mins[i]:
            mindex = i
            mini = my_mins[i]
    return mindex

def dfs(start, v, path, visited, count):
    if v in visited or count > 2:
        if start==v:
            cycles[len(cycles)] = path
        return
    visited.append(v)
    my_ind = get_indegrees(v)
    for i in my_ind:
        if new_C[v][i] == 1:
            new_path = copy.deepcopy(path)
            new_path.append(i)
            dfs(start,i,new_path, visited, count + 1)
    visited.remove(v)

def clear(num):
    for i in range (0, n):
        new_C[i][num] = 0
        new_C[num][i] = 0
        
def clashes(a, b):
    for i in range (1, cycles[a]):
        for j in range (1, cycles[b]):
            if cycles[b][j] == cycles[a][i]:
                return True
    return False
    
def choose_cycles():
    find_mins()
    index = find_mini()
    dfs(index, index, [], [], 0)
    collisions = {}
    for i in range (0, len(cycles)):
        temp = 0
        for j in range(0, len(cycles)):
            if i != j and clashes(i, j) == True:
                temp += 1
        collisions[i] = temp
    print(collisions)
    mini = n
    mindex = -1
    for i in range (0, len(collisions)):
        if collisions[i] < mini:
            mindex = i
            mini = collisions[i]
    if mindex == -1:
        return False
    else:
        saved[0] += len(cycles[mindex])
        for i in range (0, len(cycles[mindex])):
            clear(i)
        return True

cont = True
while cont:
    print('progress '+str(saved[0]))
    cont = choose_cycles()
print(saved[0])

progress 0


TypeError: 'list' object cannot be interpreted as an integer

*Explain your answer here*

## Efficiency in Two and Three-Way Exchanges
[_Efficient Kidney Exchange: Coincidence of Wants in Markets with Compatibility-Based Preferences_](http://uvammm.github.io/docs/kidneyexchange.pdf) by Alvin E. Roth, Tayfun Sönmez and M. Utku Ünver provides the bound on the number of two-way and three-way matches.

<div class="alert alert-block alert-warning">

<b>Problem 5.</b>
How does the number of matches provided by your algorithm compare with the bound in the paper? If there is a discrepency, explain why.

</div>


_Explain your answer here_

In [25]:
def tally(group):
    m = len(group)
    tal = {}
    tal[("A", "A")] = 0
    tal[("A", "B")] = 0
    tal[("A", "AB")] = 0
    tal[("A", "O")] = 0
    tal[("B", "A")] = 0
    tal[("B", "B")] = 0
    tal[("B", "AB")] = 0
    tal[("B", "O")] = 0
    tal[("O", "A")] = 0
    tal[("O", "B")] = 0
    tal[("O", "AB")] = 0
    tal[("O", "O")] = 0
    tal[("AB", "A")] = 0
    tal[("AB", "B")] = 0
    tal[("AB", "AB")] = 0
    tal[("AB", "O")] = 0
    for i in range (0, m):
        for j in range (0, m):
            if is_pair_compatible(group[i], group[j]):
                tal[(group[j]['DonorBloodType'],group[i]['ReceiverBloodType'])] += 1
    return tal
my_tally = tally(pairs)
my_tally

{('A', 'A'): 141023,
 ('A', 'B'): 0,
 ('A', 'AB'): 19459,
 ('A', 'O'): 0,
 ('B', 'A'): 0,
 ('B', 'B'): 19015,
 ('B', 'AB'): 10452,
 ('B', 'O'): 0,
 ('O', 'A'): 55593,
 ('O', 'B'): 14102,
 ('O', 'AB'): 7724,
 ('O', 'O'): 60739,
 ('AB', 'A'): 0,
 ('AB', 'B'): 0,
 ('AB', 'AB'): 5101,
 ('AB', 'O'): 0}

## Dynamic Exchange

For the final part of the assignment, we will now take the market to be dynamic: instead of having a fixed set of patients and donors, new patients arrive over time (and old patients die if they do not receive a donor kidney). This is a more realistic model of the actual kidney exchange problem. The overall goal is to maximize the overall survival of the population.

We have provided a function that simulates patient survival with a simple random draw. For each unmatch patient at the end of a time period, their probability of surviving to the next time period is given by the `ReceiverSurvivalPrb` field in the data. (We don't attempt to model patients getting sicker over time; the survival probability for each time period is independent and given by the patient's `ReceiverSurvivalPrb` field.)

In [ ]:
# This function returns True if an unmatched receiver survives until the next time period
def survival(survivalprob):
    import random
    draw = random.random()
    return draw <= survivalprob

Devlop a matching procedure that can maximize the number of matches over the six time periods in the simulated dataset accounting for patient survival.

Explain your procedure and why you think each element will help maximize the number of matches. The process is explained below:   

1. Start in TimePeriod $t$. You will implement a matching procedure and record the total number of matched agents. Those who are matched will be removed from the pool. 

2. Use the provided function `survival()` to evaluate each of the unmatched agents. If the function returns False, the agent perishes and cannot be carried over into the next month. Record the number of agents who perish and the number of agents who survive. (Note: If a patient dies, her donor is no longer willing to donate, so both the patient and donor are eliminated.)

3. Add the surviving pairs of agents to the stock for TimePeriod $t+1$

4. Repeat (1)-(4) for all time periods

We have provided a `run_simulation` function that simulates this process, given an input a matching function that implements the `match_kidneys` interface below.

Your goal is to find matches at each time period that maximize overall survival for this model. For this problem, you will want to divide your code into several functions. You should integrate your code and explanations in a way that makes it easy for a reader to understand your approach, see how you evaluate it, and connect your description to the code.

For your implementation to work in the in-class competition, it must provide this interface. Do not change function signature.

You should submit a python file (`<name>_matcher.py`) which contains this function (and any other functions you call from it). If you import libraries, please provide a `requirements.txt` with everything we will need to configure our environment to run your code.

In [15]:
def donormatches_to_tuples(indicies, patients):
    pairs = []
    for i in range(len(indicies)-1):
        pairs.append((patients[indicies[i]][0], patients[indicies[i+1]][4]))
    return pairs

def calculate_k_cycles(new_C, patients):
    comp_donors = np.zeros(n)
    matched_pairs = []
    comp_donors = compute_comp_donors(comp_donors)
    mindecies = min_index_of_array(comp_donors)
    while mindecies:
        mindex = mindecies[0]
        donor_matches = find_cycle(mindex, mindex, 3)
        if donor_matches:
            update_matrix(donor_matches)
            matched_pairs.append(donormatches_to_tuples)
#             matched_pairs += len(donor_matches) - 1
        else:
            found_no_matches[mindex] = 1
        comp_donors = compute_comp_donors(comp_donors)
        mindecies = min_index_of_array(comp_donors)
    return matched_pairs

def match_kidneys(patients, timeleft):
    """
    patients is a list of tuples like the records in patients.csv:
         (ReceiverID,ReceiverBloodType,ReceiverPRA,ReceiverSurvivalPrb,DonorID,DonorBloodType) 
    (there is no TimePeriod field). Each entry represents a patient and their (incompatible) donor-friend.
    
    timeleft is a positive integer representing the number of time periods remaining (that is, 
       when timeleft = 1, this is the final time period)
    
    The output is a list of (ReceiverID, DonorID) matches. (List of tuples)
    More specifically, the ReceiverID is from one friend-pair and the DonorID is from another friend-pair. 
    These two ID's form a transplant match, and their corresponding Donor/Patient friends they entered the pool with must
    also be a part of the exchange in the same round. 
    
    For example:
        Pretend we have two pairs of friends: (PatientAva, DonorAndy) who were not compatible, 
        and (PatientBrady, DonorBrandon) who also were not compatible. These two sets of friends enter the exchange
        because the donor of each pair can't donate to their friend.

        If PatientAva is compatible with DonorBrandon AND PatientBrady is Compatible with DonorAndy, we can form a
        two-way exchange. We would add the tuples (PatientAva, DonorBrandon) and (PatientBrady, DonorBrandon)
        to the list of matches.
        
    
    To be a valid output, all of these properties must hold:
        - No donor or recipient can appear more than once. (Donors only have one kidney to donate, recipients can only receive one)

        - A DonorID appears in the output list if only if their patient friend they volunteered for (ReceiverID) 
          that is in their patient record appears in the output list as a recipient.  
        - All (ReceiverID, DonorID) pairs in the output must be both blood and tissue compatible.
    """

    
    n = len(pairs)
    C = np.zeros((n,n))
    matched_pairs = []
    for t in range(timeleft):
        for i in range (0, n):
            for j in range (0, n):
                if survival(patients[i][3]) and survival(patients[j][3]) and is_pair_compatible(patients[i], patients[j]):
                    new_C[i][j] = 1
        print(C)
        
        matched_pairs.extend(calculate_k_cycles(C, patients))

    
    return matched_pairs 


{'TimePeriod': 0, 'ReceiverID': 583196, 'ReceiverBloodType': 'A', 'ReceiverPRA': 'Low', 'ReceiverSurvivalPrb': 0.99, 'DonorID': 306169, 'DonorBloodType': 'B'}


Note that the `match_kidneys` function is stateless. The simulation engine will maintain a list of unmatched patients that are still alive, and update the input patients to the call to `match_kidneys` for each time period to reflect the patients who survived the previous time period (either by receiving a match, which we unrealistically assume means they survive permanently!) or by not receivng a match but passing `surivial()` test, as well as newly arriving patients.

<div class="alert alert-block alert-warning">

<b>Problem 6.</b> Develop a matching algorithm that maximizes overall survival in the model where new pairs arrive each time period, and unmatched patients die with the given probability. Your code should provide clear outputs that report both the number of matches and the survival rate for each time period. You should not use any of the data from future time periods in determining what to do for the current time period.
</div>

<b>Hints</b>: Since each time period has a smaller market than the previous problems, you may want to develop strategies to thicken the market in order to increase the expected number of compatible pairs. The most obvious strategies greedily find all available matches in each time period, but better strategies may sometimes delay matches. You may be it useful to predict the likelihood that a useful donor kidney arrives in the next time period by estimating the probability a random kidney is compatible.

*Explain your answer here*

## Price-based Market For Kidney Transplantations

[_Efficient Kidney Exchange: Coincidence of Wants in Markets with Compatibility-Based Preferences_](http://uvammm.github.io/docs/kidneyexchange.pdf) discusses the design of a hypothetical competitive market for kidney transplants and derives the properties of market equilibrium. 

<div class="alert alert-block alert-warning">
    <b>Problem 7.</b> Discuss your intuition of how the prices for donors of different types should compare and how the properties of competitive prices described in the paper compares with your intuition (ideally, informed by the results of your simulations in the earlier problems).    </div>

_your answer here_

<div class="alert alert-block alert-danger">
    
Submit your Project 3 notebook by **Tuesday, 18 February 2020, 9:59pm** by creating a slack group (click “Direct Messages”) containing you and all of your teammates and the four course staff: `@cam` `@Dave` `@Denis Nekipelov` `@Kyeongtak Do`.

Submit your project to that group chat by attaching your jupyter notebook to a message (use the “+” at the left of the message entry field to attach a file).
</div>